In [1]:
import os
import time

In [11]:
import pandas as panda
df=panda.read_csv('vgsales.csv')
df

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


In [13]:
# Reading data with Dask

In [6]:
from dask import dataframe as dd
start = time.time()
dask_df = dd.read_csv('vgsales.csv')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.020991086959838867 sec


In [ ]:
# Read in the data with Modin and Ray

In [3]:
import modin.pandas as pd
import ray
ray.shutdown()
ray.init()
start = time.time()
df = pd.read_csv('vgsales.csv')
end = time.time()
print("Read csv with modin and ray: ",(end-start),"sec")

2022-09-10 12:40:56,383	INFO services.py:1456 -- View the Ray dashboard at http://127.0.0.1:8265


Read csv with modin and ray:  5.6590025424957275 sec


In [ ]:
# Here Dask is better than Pandas, Modin and Ray, with the least reading time of  0.019 sec


In [5]:
from dask import dataframe as dd
df = dd.read_csv('vgsales.csv',delimiter=',')

In [7]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 11 entries, Rank to Global_Sales
dtypes: object(4), float64(5), int64(2)

In [9]:
#No. of Rows
len(df.columns)

11

In [10]:
# remove special character
df.columns=df.columns.str.replace('[#,@,&]','')

In [11]:
#To remove white space from columns
df.columns = df.columns.str.replace(' ', '')

In [12]:
data=df.columns
data

Index(['Rank', 'Name', 'Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'],
      dtype='object')

In [ ]:
# Validation


In [13]:
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

In [14]:
%%writefile utility.py

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.load(stream, Loader=yaml.Loader)
        except yaml.YAMLError as exc:
            logging.error(exc)

def col_header_val(df,table_config):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing utility.py


In [25]:
%%writefile store.yaml
file_type: csv
dataset_name: file
file_name: Rate
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns:
    - Rank
    - Name
    - Platform
    - Year
    - Genre
    - Publisher
    - NA_Sales
    - EU_Sales
    - JP_Sales
    - Other_Sales
    - Global_Sales

Overwriting store.yaml
